In [ ]:
import pandas as pd

# Echonest audio features
## 1. Premise
The hyper-parameters and results for unbalanced classifiers are taken  from the notebooks:
- bin-genre_decision-tree_KNN
- multi-genre_decision-tree-KNN
- popularity_decision-tree_KNN
Below we report their score and AUC curve for comparing them with the balanced data.